# Neuroscape playground

In [1]:
import haiku as hk
import jax
import optax
import jax.numpy as jnp
from jax import pmap, vmap, grad, jit
import numpy as np
from typing import Tuple, List, Dict
from tqdm import tqdm
import wandb
from functools import partial
from src.train import update, evaluate
from src.model import init, forward, loss_fn, network_fn
from src.utils import get_args_and_config
from src.data import get_loaders

## data

In [2]:
opt = optax.adam(1e-3)
args, config = get_args_and_config()
folds, _ = get_loaders(args, config)

100%|██████████| 9840/9840 [01:16<00:00, 128.34it/s]


In [3]:
rng = hk.PRNGSequence(jax.random.PRNGKey(42))

# types
Fold = Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]
Batch = Fold


# functions
def train(folds: List[Fold], config: Dict) -> List[hk.Params]:
    """train function"""
    # TODO: parallelize using pmap or vmap
    config['group_name'] = wandb.util.generate_id()
    folds_lst = [make_fold(folds, fold) for fold in range(len(folds))]  # (train_data, val_data) list
    params_lst = [init(next(rng), folds[0][0], folds[0][1]) for _ in range(len(folds))]
    train_fold = partial(train_fold_fn, config=config)
    for idx, (params, fold) in enumerate(zip(params_lst, folds_lst)):
        params_lst[idx] = train_fold(params, fold)
    return params_lst


def train_fold_fn(params, fold, config: Dict) -> hk.Params:
    """train_fold function"""
    train_data, val_data = fold
    config['param_count'] = count_params(params)
    wandb.init(project="neuroscope", entity='syrkis', config=config, group=config['group_name'])
    opt_state = opt.init(params)
    for step in range(config['n_steps']):
        img, cat, fmri = get_batch(train_data, config['batch_size'])
        params, opt_state = update(params, img, cat, fmri, opt_state)
        if step % (config['n_steps'] // 100) == 0:
            metrics = evaluate(params, train_data, val_data, get_batch)
            wandb.log(metrics, step=step)
    wandb.finish()
    return params


def get_batch(fold: Fold, batch_size: int) -> Batch:
    """get a batch from a split"""
    img, cat, fmri = fold
    idx = np.random.randint(0, img.shape[0], batch_size)
    return img[idx], cat[idx], fmri[idx]


def make_fold(folds: List[Fold], fold: int) -> Batch:
    """make a fold from a list of folds"""
    train_imgs = [f[0] for f in folds[:fold] + folds[fold + 1:]]
    train_cats = [f[1] for f in folds[:fold] + folds[fold + 1:]]
    train_fmris = [f[2] for f in folds[:fold] + folds[fold + 1:]]
    train_data = tuple(map(jnp.concatenate, [train_imgs, train_cats, train_fmris]))
    return train_data, folds[fold]


@jit
def update(params: hk.Params, img: jnp.ndarray, cat: jnp.ndarray, fmri: jnp.ndarray, opt_state: optax.OptState) -> Tuple[hk.Params, optax.OptState]:
    grads = grad(loss_fn)(params, img, cat, fmri)
    updates, opt_state = opt.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, opt_state

def count_params(params):
    return sum([np.prod(v.shape) for v in jax.tree_util.tree_leaves(params)])

params_lst = train(folds, config)

AttributeError: module 'haiku' has no attribute 'Dropout'